# Configurartion

In [ ]:
!pip install tqdm notebook==7.1.2 openai elasticsearch pandas scikit-learn ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:

In [ ]:
from google.colab import userdata
import os
from openai import OpenAI

# acquire API key from Colab userdata and set as environment variable
os.environ['OPENAI_API_KEY'] = userdata.get('oaitest')

# create OpenAI client rather than pass the the API key directly
client = OpenAI()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-11-05 15:03:05--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-11-05 15:03:06 (82.6 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [ ]:
!cat minsearch.py

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np


class Index:
    """
    A simple search index using TF-IDF and cosine similarity for text fields and exact matching for keyword fields.

    Attributes:
        text_fields (list): List of text field names to index.
        keyword_fields (list): List of keyword field names to index.
        vectorizers (dict): Dictionary of TfidfVectorizer instances for each text field.
        keyword_df (pd.DataFrame): DataFrame containing keyword field data.
        text_matrices (dict): Dictionary of TF-IDF matrices for each text field.
        docs (list): List of documents indexed.
    """

    def __init__(self, text_fields, keyword_fields, vectorizer_params={}):
        """
        Initializes the Index with specified text and keyword fields.

        Args:
            text_fields (list): List of text field names to index.
            ke

In [ ]:
import requests

url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py"
response = requests.get(url)

with open("minsearch.py", "w") as f:
    f.write(response.text)

print("File contents:")
print(response.text[:500])  # Print the first 500 characters of the file

File contents:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np


class Index:
    """
    A simple search index using TF-IDF and cosine similarity for text fields and exact matching for keyword fields.

    Attributes:
        text_fields (list): List of text field names to index.
        keyword_fields (list): List of keyword field names to index.
        vectorizers (dict): Dictionary of TfidfVectorize


In [ ]:
import minsearch

In [ ]:
import json
import requests

In [ ]:
url = "https://raw.githubusercontent.com/AD2000X/llm-zoomcamp/refs/heads/main/01-intro/documents.json"
response = requests.get(url)

In [ ]:
if response.status_code == 200:
    doc_raw = json.loads(response.text)
    print("JSON data loaded successfully")
    print(f"Number of items in the JSON: {len(doc_raw)}")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

JSON data loaded successfully
Number of items in the JSON: 3


In [ ]:
documents = []

for course_dict in doc_raw:
  for doc in course_dict['documents']:
    doc['course'] = course_dict['course']
    documents.append(doc)

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
index.fit(documents)

# 1.5 - The RAG Flow Cleaning and Modularizing Code

### build search

In [ ]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10

    )

    return results

### build prompt

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ]
    )
    return response.choices[0].message.content

### ---trial run before build RAG

In [ ]:
# num_results = 2

query = 'How do I run kafka?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [ ]:
search_results

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [ ]:
answer

'To run Kafka, you can execute the following command in the project directory:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the appropriate jar file name in your project.'

In [ ]:
# num_results = 2

query = 'the course has alreaady started, can I still enroll?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [ ]:
search_results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [ ]:
answer

"Yes, you can still enroll in the course even if it has already started. You're eligible to submit the homework assignments. However, be mindful of the deadlines for the final projects and try not to leave everything for the last minute."

In [ ]:
# change num_results to 10

query = 'How do I run kafka?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [ ]:
answer

'To run Kafka, if you are using Java, you need to execute the following command in the project directory:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you are using Python and facing a "Module not found" error, it\'s recommended to create a virtual environment and run the `requirements.txt` file and the Python files within that environment. Here\'s how you can set up and activate the virtual environment:\n\n1. Create a virtual environment (run only once):\n   ```bash\n   python -m venv env\n   ```\n\n2. Activate the virtual environment:\n   - On MacOS or Linux:\n     ```bash\n     source env/bin/activate\n     ```\n   - On Windows:\n     ```bash\n     env\\Scripts\\activate\n     ```\n\n3. Install the necessary packages:\n   ```bash\n   pip install -r ../requirements.txt\n   ```\n\nTo deactivate the virtual environment, simply run:\n\n```bash\ndeactivate\n```\n\nPlease make sure your Docker images are all up and 

### ---trial run ends here

### def RAG

In [ ]:
# num_results = 10, line 14

query = 'How do I run kafka?' # move query out of the def rag(query)

def rag(query):
    search_results = search(query)  # search can be changed to other search engine
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)  # llm can be changed to other llm

    return answer

In [ ]:
rag(query)

'To run Kafka, navigate to your project directory and execute the following command in the terminal:\n\n```shell\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nThis command is used to run the Java Kafka producer, consumer, kstreams, etc.'

In [ ]:
rag('the course has alreaady started, can I still enroll?')

'Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homework, but be sure to comply with the deadlines for submitting final projects, so avoid leaving everything to the last minute.'

### ---examine serach_reasults and answer in rag

In [ ]:
# num_results = 10, line 14

def rag(query):
    search_results = search(query)

    print("Search Results:")
    for i, result in enumerate(search_results, 1):
        print(f"Result {i}:")
        print(f"Q: {result['question']}")
        print(f"A: {result['text'][:100]}...")  # Print only the first 100 characters of the answer.
        print()

    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

# call rag function
result = rag(query)
print("Final Answer:", result)

Search Results:
Result 1:
Q: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
A: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org...

Result 2:
Q: Module “kafka” not found when trying to run producer.py
A: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in ...

Result 3:
Q: How do I install the necessary dependencies to run the code?
A: Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do thi...

Result 4:
Q: Python Kafka: ./build.sh: Permission denied Error
A: Run this command in terminal in the same directory (/docker/spark):
chmod +x build.sh...

Result 5:
Q: How to fix the error "ModuleNotFoundError: No module named 'kafka.vendor.six.moves'"?
A: According to https://github.com/dpkp/kafka-python/
“DUE TO ISSUES WITH RELEASES, IT IS SUGGESTED TO ...

Result 6:
Q: kafka.errors.NoBrokersAvailable: NoBrokersAvailable
A: If you 

In [ ]:
query = 'How do I run kafka?'
search_results = search(query)

print("\nConcise Search Results:")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Q: {result['question']}")
    print(f"A: {result['text'][:100]}...")  # 只打印答案的前100個字符
    print()


Concise Search Results:
Result 1:
Q: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
A: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org...

Result 2:
Q: Module “kafka” not found when trying to run producer.py
A: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in ...

Result 3:
Q: How do I install the necessary dependencies to run the code?
A: Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do thi...

Result 4:
Q: Python Kafka: ./build.sh: Permission denied Error
A: Run this command in terminal in the same directory (/docker/spark):
chmod +x build.sh...

Result 5:
Q: How to fix the error "ModuleNotFoundError: No module named 'kafka.vendor.six.moves'"?
A: According to https://github.com/dpkp/kafka-python/
“DUE TO ISSUES WITH RELEASES, IT IS SUGGESTED TO ...

Result 6:
Q: kafka.errors.NoBrokersAvailable: NoBrokersAvailable
A

In [ ]:
# 或者直接調用 rag 函數
result = rag(query)

Search Results:
Result 1:
Q: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
A: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org...

Result 2:
Q: Module “kafka” not found when trying to run producer.py
A: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in ...

Result 3:
Q: How do I install the necessary dependencies to run the code?
A: Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do thi...

Result 4:
Q: Python Kafka: ./build.sh: Permission denied Error
A: Run this command in terminal in the same directory (/docker/spark):
chmod +x build.sh...

Result 5:
Q: How to fix the error "ModuleNotFoundError: No module named 'kafka.vendor.six.moves'"?
A: According to https://github.com/dpkp/kafka-python/
“DUE TO ISSUES WITH RELEASES, IT IS SUGGESTED TO ...

Result 6:
Q: kafka.errors.NoBrokersAvailable: NoBrokersAvailable
A: If you 

### ---examine serach_reasults and answer in rag ends here

# 1.6 Elasticsearch

In [ ]:
!pip install elasticsearch

In [ ]:
import logging
import os
from elasticsearch import Elasticsearch
from google.colab import userdata
from getpass import getpass

# 嘗試從 Colab 的環境變數中取得 Elastic Cloud ID 和 API Key
ELASTIC_CLOUD_ID = userdata.get('ELASTIC_CLOUD_ID')
ELASTIC_API_KEY = userdata.get('ELASTIC_API_KEY')

# 如果環境變數中沒有找到，則請求用戶輸入
if not ELASTIC_CLOUD_ID:
    ELASTIC_CLOUD_ID = getpass("輸入 Elastic Cloud ID: ")

if not ELASTIC_API_KEY:
    ELASTIC_API_KEY = getpass("輸入 Elastic Api Key: ")

try:
    # Create the client
    es_client = Elasticsearch(
        cloud_id=ELASTIC_CLOUD_ID,
        api_key=ELASTIC_API_KEY,
        timeout=60  # 增加超時時間
    )

    # Check connection
    if es_client.ping():
        print('Successfully connected to Elasticsearch')
        print(es_client.info())
    else:
        print('Could not connect to Elasticsearch')

except Exception as e:
    print(f"Connection error: {str(e)}")

<ipython-input-32-98a0701a3ba1>:20: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es_client = Elasticsearch(


Successfully connected to Elasticsearch
{'name': 'instance-0000000001', 'cluster_name': 'bd5c04e72b0c4673b4cbd3db98fdb86b', 'cluster_uuid': 'K11-nm1NQeu_GpxDE5TTJg', 'version': {'number': '8.14.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd55f984299e0e88dee72ebd8255f7ff130859ad0', 'build_date': '2024-07-07T22:04:49.882652950Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [ ]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
es_client.info()

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': 'bd5c04e72b0c4673b4cbd3db98fdb86b', 'cluster_uuid': 'K11-nm1NQeu_GpxDE5TTJg', 'version': {'number': '8.14.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd55f984299e0e88dee72ebd8255f7ff130859ad0', 'build_date': '2024-07-07T22:04:49.882652950Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Index settings:

In [ ]:
# 定義索引設置
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

# # 創建索引
# result = es_client.indices.create(index=index_name, body=index_settings)
# print(f"創建索引結果: {result}")

# # 驗證索引創建是否成功
# if result.get('acknowledged'):
#     print("索引創建成功！")
# else:
#     print("索引創建失敗。")

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/E5SZf1OnSsy0mHa7sOxgAA] already exists')

In [ ]:
try:
    exists = es_client.indices.exists(index="course-questions")
    print(f"索引檢查結果: {'已存在' if exists else '不存在'}")

    if not exists:
        result = es_client.indices.create(index="course-questions", body=index_settings)
        print(f"索引創建狀態: {'成功' if result.get('acknowledged') else '失敗'}")
except Exception as e:
    print(f"錯誤: {str(e)}")

索引檢查結果: 已存在


In [ ]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Query:

In [ ]:
query

'How do I run kafka?'

In [ ]:
query = 'I just discover the course. Can I still join it?'

In [ ]:
search_query = {
    "size": 5,  # we will only get 5 answers
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],  # question^3: question is 3 times more important than text or section
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
es_client.search(index=index_name, body=search_query)

ObjectApiResponse({'took': 24, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 405, 'relation': 'eq'}, 'max_score': 72.849266, 'hits': [{'_index': 'course-questions', '_id': 'ASTc_JIByjD8izo6PZw9', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'BiTc_JIByjD8izo6QJyv', '_score': 54.057133, '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next 

In [ ]:
response = es_client.search(index=index_name, body=search_query)

In [ ]:
response

ObjectApiResponse({'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 405, 'relation': 'eq'}, 'max_score': 72.849266, 'hits': [{'_index': 'course-questions', '_id': 'ASTc_JIByjD8izo6PZw9', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'BiTc_JIByjD8izo6QJyv', '_score': 54.057133, '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next c

In [ ]:
response ['hits']

# ['hits'] outputs' include metadata: {'total': {'value': 405, 'relation': 'eq'}, 'max_score': 72.849266,

{'total': {'value': 405, 'relation': 'eq'},
 'max_score': 72.849266,
 'hits': [{'_index': 'course-questions',
   '_id': 'ASTc_JIByjD8izo6PZw9',
   '_score': 72.849266,
   '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
    'section': 'General course-related questions',
    'question': 'Course - Can I still join the course after the start date?',
    'course': 'data-engineering-zoomcamp'}},
  {'_index': 'course-questions',
   '_id': 'BiTc_JIByjD8izo6QJyv',
   '_score': 54.057133,
   '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
    'section': 'G

In [ ]:
response ['hits']['hits']

# ['hits'['hits'] outputs' don't include metadata: {'total': {'value': 405, 'relation': 'eq'}, 'max_score': 72.849266,

[{'_index': 'course-questions',
  '_id': 'ASTc_JIByjD8izo6PZw9',
  '_score': 72.849266,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'BiTc_JIByjD8izo6QJyv',
  '_score': 54.057133,
  '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
   'section': 'General course-related questions',
   'question': 'Course - Can I follow the course after 

In [ ]:
# we need to go into the source
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

# # Suppose hit is a dictionary structured like this:
# hit = {
#     '_index': 'course-questions',     # Other data
#     '_id': 'ASTc_JIByjD8izo6PZw9',   # Other data
#     '_score': 72.849266,              # Other data
#     '_source': {                      # <- The data we need is here
#         'text': "Yes, even if...",
#         'section': "General course-related questions",
#         'question': "Course - Can I still...",
#         'course': "data-engineering-zoomcamp"
#     }
# }

# # hit['_source'] extracts all the data inside _source:
# source_data = hit['_source']
# # source_data now contains:
# {
#     'text': "Yes, even if...",
#     'section': "General course-related questions",
#     'question': "Course - Can I still...",
#     'course': "data-engineering-zoomcamp"
# }

# # So when we write:
# result_docs.append(hit['_source'])

# # It is equivalent to:
# result_docs.append({
#     'text': "Yes, even if...",
#     'section': "General course-related questions",
#     'question': "Course - Can I still...",
#     'course': "data-engineering-zoomcamp"
# })


In [1]:
# # Create an empty list
# fruits = []
# print("Initial empty list:", fruits)  # Output: []

# # Add a single fruit using append
# fruits.append("Apple")
# print("After adding Apple:", fruits)  # Output: ['Apple']

# # Add another fruit
# fruits.append("Banana")
# print("After adding Banana:", fruits)  # Output: ['Apple', 'Banana']

# # Add one more fruit
# fruits.append("Orange")
# print("After adding Orange:", fruits)  # Output: ['Apple', 'Banana', 'Orange']


Initial empty list: []
After adding Apple: ['Apple']
After adding Banana: ['Apple', 'Banana']
After adding Orange: ['Apple', 'Banana', 'Orange']


In [ ]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

# Put search function together and test

In [ ]:
def elastic_search(query):

  search_query = {
      "size": 5,  # we will only get 5 answers
      "query": {
          "bool": {
              "must": {
                  "multi_match": {
                      "query": query,
                      "fields": ["question^3", "text", "section"],  # question^3: question is 3 times more important than text or section
                      "type": "best_fields"
                  }
              },
              "filter": {
                  "term": {
                      "course": "data-engineering-zoomcamp"
                  }
              }
          }
      }
  }

  response = es_client.search(index=index_name, body=search_query)

  result_docs = []

  for hit in response['hits']['hits']:
      result_docs.append(hit['_source'])

  return result_docs

In [ ]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

# # Put RAG and search function together and test

In [ ]:
def elastic_search(query):

  search_query = {
      "size": 5,  # we will only get 5 answers
      "query": {
          "bool": {
              "must": {
                  "multi_match": {
                      "query": query,
                      "fields": ["question^3", "text", "section"],  # question^3: question is 3 times more important than text or section
                      "type": "best_fields"
                  }
              },
              "filter": {
                  "term": {
                      "course": "data-engineering-zoomcamp"
                  }
              }
          }
      }
  }

  response = es_client.search(index=index_name, body=search_query)

  result_docs = []

  for hit in response['hits']['hits']:
      result_docs.append(hit['_source'])

  return result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)  # search can be changed to other search engine
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)  # llm can be changed to other llm
    return answer

In [ ]:
rag(query)

"Yes, you can still join the course even if you've just discovered it. You are eligible to submit the homeworks even without registering. However, keep in mind that there will be deadlines for turning in the final projects, so don't procrastinate."